# Resource Estimation for Double-factorized Chemistry

In this notebook we evaluate the physical resource estimates of using the
so-called _double-factorized qubitization_ algorithm described in [[Phys. Rev.
Research 3, 033055 (2021)](https://doi.org/10.1103/PhysRevResearch.3.033055)] to
calculate the energy of a user provided Hamiltonian to chemical accuracy of 1
mHa.  The Hamiltonian is provided in terms of an FCIDUMP file that is accessible
via an HTTPS URI.

The _qubitization_ approach is based on quantum phase estimation, but instead of
constructing the standard $U = \exp{(-i H/\alpha)}$ from the Hamiltonian matrix
$H$, one takes $U = \exp{(-i \sin^{-1} (H/\alpha))}$, which can typically be
implemented with fewer resources. Using _double-factorization_, $H$ is
represented compactly through a combination of a judicious choice of orbitals
and compression. The tolerated total error budget is $\epsilon = 0.01$,
corresponding to $1\%$.

## Getting started

We connect to the Azure Quantum workspace by creating a new workspace.  This workspace is then used to create an instance to the Resource Estimator.

In [ ]:
from azure.quantum import Workspace
from azure.quantum.target.microsoft import MicrosoftEstimator
from azure.quantum.chemistry import df_chemistry

workspace = Workspace(
    resource_id="",
    location=""
)

estimator = MicrosoftEstimator(workspace)

# Configuring a resource estimation job

We start by creating a parameter instance for the resource estimator, which allows us to configure all parameters associated to the estimation job.  In this scenario, we want to evaluate estimates for six different qubit parameter configurations, therefore we set the number of items in the job to 6.

In [ ]:
params = estimator.make_params(num_items=6)

Next we are providing a link to the FCIDUMP file that describes the Hamiltonian, which is passed via a URI. For example, you can choose some of the following URIs:

| URI                                          | Instance name         | Description |
|----------------------------------------------|-----------------------|-------------|
| https://aka.ms/fcidump/XVIII-cas4-fb-64e-56o | XVIII-cas4-fb-64e-56o |             |
| https://aka.ms/fcidump/nitrogenase-54e-54o   | nitrogenase-54e-54o   |             |
| https://aka.ms/fcidump/fe2s2-10e-40o         | fe2s2-10e-40o         |             |
| https://aka.ms/fcidump/polyyne-24e-24o       | polyyne-24e-24o       |             |
| https://aka.ms/fcidump/n2-10e-8o             | n2-10e-8o             |             |

You can also pass your own FCIDUMP files via 
* [raw links to files in Github](https://docs.github.com/repositories/working-with-files/using-files/viewing-a-file#viewing-or-copying-the-raw-file-content) repositories (see how to [add files to Github repositories](https://docs.github.com/repositories/working-with-files/managing-files/creating-new-files))
* [files on Github gists](https://docs.github.com/get-started/writing-on-github/editing-and-sharing-content-with-gists/creating-gists)
* [files in Azure Blob Storage](https://learn.microsoft.com/azure/storage/blobs/storage-blobs-introduction) using [SAS tokens](https://learn.microsoft.com/azure/cognitive-services/translator/document-translation/how-to-guides/create-sas-tokens?tabs=Containers#create-sas-tokens-in-the-azure-portal)

The URI is passed to the parameters as so called file URI with the name `"fcidumpUri"`:

In [ ]:
params.file_uris["fcidumpUri"] = "https://aka.ms/fcidump/XVIII-cas4-fb-64e-56o"

The quantum algorithms requires a total accuracy if 0.01, i.e., 1%, in order to obtain a chemical accuracy of 1 mHa.  We can instruct the resource estimator to use a total error budget of 0.01, which is distributed to all possible sub components in the execution of the quantum algorithm that may fail. (More details on the error budget can be found in the [Azure Quantum documentation](https://learn.microsoft.com/azure/quantum/overview-resources-estimator#error-budget).)

In [ ]:
params.error_budget = 0.01

Finally, we are specifying the qubit parameters.  Here we are choosing all six pre-defined qubit parameter models.  These are four gate-based models with operation times in the microsecond and nanosecond regime, as well as assumptions on their physical error rates of $10^{-3}$ and $10^{-4}$, respectively.  The other two are Majorana based models with operation times in the nanosecond regime and physical error rates of $10^{-4}$ and $10^{-6}$.  For the Majorana based models we assume a Floquet code as QEC scheme.  More details on these parameters and assumptions, as well as how to customize these, can be found in the [Azure Quantum documentation](https://learn.microsoft.com/azure/quantum/overview-resources-estimator).

In [ ]:
params.items[0].qubit_params.name = "qubit_gate_us_e3"
params.items[1].qubit_params.name = "qubit_gate_us_e4"
params.items[2].qubit_params.name = "qubit_gate_ns_e3"
params.items[3].qubit_params.name = "qubit_gate_ns_e4"
params.items[4].qubit_params.name = "qubit_maj_ns_e4"
params.items[4].qec_scheme.name = "floquet_code"
params.items[5].qubit_params.name = "qubit_maj_ns_e6"
params.items[5].qec_scheme.name = "floquet_code"

The parameters are now all set up, and we are ready to go to submit the resource estimation job. As quantum program, we are using the double-factorization based quantum chemistry algorithm, which is provided via the `df_chemistry` function.  The execution of this cell may take a few minutes depending on program size.  Once the job is finished, we are obtaining the results.

In [ ]:
job = estimator.submit(df_chemistry(), input_params=params)
results = job.get_results()

## Analyzing the results

Now that the results have been computed, we display them in a summary table. For this purpose we are creating a reusable `dashboard` function that is creating an HTML display from a pandas data frame and the resource estimation tables.

In [ ]:
labels = ["Gate-based µs, 10⁻³", "Gate-based µs, 10⁻⁴", "Gate-based ns, 10⁻³", "Gate-based ns, 10⁻⁴", "Majorana ns, 10⁻⁴", "Majorana ns, 10⁻⁶"]

def dashboard(results):
    def get_row(result):
        # Extract raw data from result dictionary
        logical_qubits = result["physicalCounts"]["breakdown"]["algorithmicLogicalQubits"]
        logical_depth = result["physicalCounts"]["breakdown"]["logicalDepth"]
        num_tstates = result["physicalCounts"]["breakdown"]["numTstates"]
        code_distance = result["logicalQubit"]["codeDistance"]
        num_tfactories = result["physicalCounts"]["breakdown"]["numTfactories"]
        tfactory_fraction = (result["physicalCounts"]["breakdown"]["physicalQubitsForTfactories"] / result["physicalCounts"]["physicalQubits"]) * 100
        physical_qubits = result["physicalCounts"]["physicalQubits"]
        runtime = result["physicalCounts"]["runtime"]

        # Format some entries
        logical_depth_formatted = f"{logical_depth:.1e}"
        num_tstates_formatted = f"{num_tstates:.1e}"
        tfactory_fraction_formatted = f"{tfactory_fraction:.1f}%"
        physical_qubits_formatted = f"{physical_qubits / 1e6:.2f}M"

        # Make runtime human readable; we find the largest units for which the
        # runtime has a value that is larger than 1.0.  For that unit we are
        # rounding the value and append the unit suffix.
        units = [("nanosecs", 1), ("microsecs", 1000), ("millisecs", 1000), ("secs", 1000), ("mins", 60), ("hours", 60), ("days", 24), ("years", 365)]
        runtime_formatted = runtime
        for idx in range(1, len(units)):
            if runtime_formatted / units[idx][1] < 1.0:
                runtime_formatted = f"{round(runtime_formatted) % units[idx][1]} {units[idx - 1][0]}"
                break
            else:
                runtime_formatted = runtime_formatted / units[idx][1]

        # special case for years
        if isinstance(runtime_formatted, float):
            runtime_formatted = f"{round(runtime_formatted)} {units[-1][0]}"

        # Append all extracted and formatted data to data array
        return (logical_qubits, logical_depth_formatted, num_tstates_formatted, code_distance, num_tfactories, tfactory_fraction_formatted, physical_qubits_formatted, runtime_formatted)

    data = [get_row(results.data(index)) for index in range(len(results))]

    # Create data frame with explicit column names and configuration names extracted from array
    import pandas as pd
    df = pd.DataFrame(data, columns=["Logical qubits", "Logical depth", "T states", "Code distance", "T factories", "T factory fraction", "Physical qubits", "Physical runtime"], index=labels)

    return df

dashboard(results)

Each row corresponds to one of the six qubit parameter configurations, where the first column shows a textual description for the model.  The next three columns show technology-independent resources, which are the number of logical qubits, the logical depth, which is the number of logical operations performed in sequence, as well as the number of T states that are consumed by the logical operations.  T states originate from complex operations in the quantum algorithm, e.g., Toffoli gates or rotation gates.

Next, the code distance indicates the error correction overhead to guarantee a sufficient logical error rate for the logical operations.  The number of T factories indicates how many T factories are executed in parallel to produce the total number of T states.  The T factory fraction describes the percentage of the number of qubits that are used to execute T factories, the rest is used to execute the logical operations of the algorithm.  Finally, the last two columns show the total number of physical qubits and the wall clock runtime to execute the quantum algorithm given the assumed qubit parameters.

### Detailed resource estimates

We can also have a more detailed look into the resource estimates.  Here we show the details for the last configuration (index 5).  The output is a table with the overall physical resource counts. You can further inspect more details about the resource estimates by collapsing various groups which have more information. For example, if you collapse the Logical qubit parameters group, you can see how the overhead to represent a logical qubit using physical qubits is derived. The last group shows the physical qubit properties that were assumed for this estimation.

In [ ]:
results[5]

We can also compare different configurations.  In this case we compare the gate-based nanosecond model with the Majorana based model for an error rate of $10^{-4}$.  These correspond to indices 3 and 4, not that intervals in Python are half-open.

In [ ]:
results[3:5]

## Next steps

We hope that this notebook was helpful to you in order to estimate the quantum computing requirements to calculate the energy of a Hamiltonian.  Here are some suggestions for next steps:
* Try to estimate some custom FCIDUMP files
* Investigate the details of resource estimation by exploring the detailed resource estimation tables
* Modify the assumptions on the target quantum computer by providing custom qubit parameters
* Check out the other resource estimation sample notebooks in the Azure Quantum sample gallery